#Projet IA 2023 - Prédire le prix de réservation d'hôtel

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Importation des données

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
df = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/DATA/df_complet.csv')

In [39]:
df.head()

,hotel_id,price,stock,language,city,date,mobile,avatar_id,group,brand,parking,pool,children_policy
0,58,322,1,hungarian,copenhagen,2,0,976,Yin Yang,Royal Lotus,1,0,2
1,402,128,0,hungarian,copenhagen,2,0,976,Independant,Independant,1,0,0
2,506,148,0,hungarian,copenhagen,2,0,976,Independant,Independant,1,0,0
3,930,120,0,hungarian,copenhagen,2,0,976,Independant,Independant,0,0,0
4,443,145,1,hungarian,copenhagen,2,0,976,Accar Hotels,Navatel,0,0,0


In [40]:
df_date = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/DATA/df_date_glob.csv')

In [41]:
df_init = pd.concat([df,df_date])

In [42]:
df_init

,hotel_id,price,stock,language,city,date,mobile,avatar_id,group,brand,parking,pool,children_policy
0,58,322,1,hungarian,copenhagen,2,0,976,Yin Yang,Royal Lotus,1,0,2
1,402,128,0,hungarian,copenhagen,2,0,976,Independant,Independant,1,0,0
2,506,148,0,hungarian,copenhagen,2,0,976,Independant,Independant,1,0,0
3,930,120,0,hungarian,copenhagen,2,0,976,Independant,Independant,0,0,0
4,443,145,1,hungarian,copenhagen,2,0,976,Accar Hotels,Navatel,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194171,521,95,0,finnish,valletta,1,1,267774,Accar Hotels,Ibas,1,0,0
194172,749,136,0,finnish,valletta,1,1,267774,Independant,Independant,0,0,0
194173,639,301,1,finnish,valletta,1,1,267774,Yin Yang,Royal Lotus,0,1,0
194174,905,181,0,finnish,valletta,1,1,267774,Yin Yang,Ardisson,0,0,0


Ajout colonne cluster

In [43]:
df44 = df[df.date==44]
print(len(df44))
dic = {'hotel_id':[],
       'price':[]} # prix moyen

for h in np.unique(df44['hotel_id']):
  dic['hotel_id'].append(h)
  dic['price'].append(df44[df44.hotel_id==h]['price'].mean())

27025


In [44]:
new_df = pd.DataFrame(dic)
new_df

,hotel_id,price
0,0,167.600000
1,1,85.980769
2,2,81.259259
3,3,61.615385
4,4,38.560000
...,...,...
994,994,154.846154
995,995,184.942308
996,996,95.714286
997,997,109.962963


In [45]:
val = np.array(new_df['price']).reshape(-1,1)
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=5, random_state=0).fit(val)
clust = kmeans.labels_

np.shape(clust)

(999,)

In [46]:
new_df=new_df.assign(cluster=clust)
new_df.head()

,hotel_id,price,cluster
0,0,167.600000,1
1,1,85.980769,2
2,2,81.259259,2
3,3,61.615385,2
4,4,38.560000,2


In [72]:
df = df_init

# ajout colonne cluster
df = df.merge(new_df[['hotel_id','cluster']], on='hotel_id', how='left')

# Transformation des variables qualitatives en quantitatives
dummies_language = pd.get_dummies(df['language'], prefix= 'Language')
dummies_city = pd.get_dummies(df['city'], prefix= 'City')
dummies_group = pd.get_dummies(df['group'], prefix= 'Group')
dummies_brand = pd.get_dummies(df['brand'], prefix= 'Brand')

df = pd.concat([df, dummies_language, dummies_city, dummies_group, dummies_brand], axis=1)
df.drop(['language', 'city', 'group', 'brand', 'avatar_id'], axis=1, inplace=True)

### Création jeu d'entraînement et de test

In [48]:
df = df[df.date<60] # à faire pour améliorer la prédiction

In [73]:
Y_train = df['price']
X_train = df
print(len(X_train.columns))
X_train.drop(['price'], axis=1, inplace=True)
#X_train.drop(['hotel_id'], axis=1, inplace=True) # on retire les hotel_id
print(len(X_train.columns))

67
66


In [74]:
X_train

,hotel_id,stock,date,mobile,parking,pool,children_policy,cluster,Language_austrian,Language_belgian,...,Brand_Ibas,Brand_Independant,Brand_J.Halliday Inn,Brand_Marcure,Brand_Morriot,Brand_Navatel,Brand_Quadrupletree,Brand_Royal Lotus,Brand_Safitel,Brand_Tripletree
0,58,1,2,0,1,0,2,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,402,0,2,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,506,0,2,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,930,0,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,443,1,2,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675231,521,0,1,1,1,0,0,2,0,0,...,1,0,0,0,0,0,0,0,0,0
675232,749,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
675233,639,1,1,1,0,1,0,4,0,0,...,0,0,0,0,0,0,0,1,0,0
675234,905,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Retrait des date > 30

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.7, random_state=42)

##XGboost

In [51]:
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score

# Mesures de performance
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn import metrics

In [30]:
model = GradientBoostingRegressor()

# Définition des différents paramètres à tester
n_estimators = [100] # [50, 70, 80, 90] # Nombre d'arbres, plus il y a d'arbres plus l'algo a tendance à surapprendre sur les données
loss = ['squared_error', 'absolute_error'] # 
learning_rate = [0.01,0.1] # taux d'apprentissage => permet d'éviter le surapprentissage
criterion = ['friedman_mse', 'squared_error','mse'] #
max_features = ['sqrt', 'log2'] #
max_depth = [5,10,15]

# Définition de la grille de recherche
grid = dict(n_estimators = n_estimators, loss = loss, learning_rate = learning_rate,  criterion = criterion, max_features = max_features, max_depth = max_depth)
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 3, random_state = 1)

grid_search = GridSearchCV(estimator = model, 
                        param_grid = grid, 
                        n_jobs = -1, 
                        cv = cv, 
                        scoring = metrics.mean_squared_error, 
                        error_score = 0)

grid_result = grid_search.fit(X_train, y_train)

# Résumé des résultats
print("Meilleur taux : %f avec pour paramètres %s" % (grid_result.best_score_, grid_result.best_params_))
best_param_gbm = grid_result.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Meilleur taux : 0.000000 avec pour paramètres {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 100}


In [32]:
grid_search

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
             error_score=0, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error', 'mse'],
                         'learning_rate': [0.01, 0.1],
                         'loss': ['squared_error', 'absolute_error'],
                         'max_depth': [5, 10, 15],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100]},
             scoring=<function mean_squared_error at 0x7f217f91ec10>)

'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 100

In [ ]:
# Affiche le détails pour les autres paramètres
moy = grid_result.cv_results_['mean_train_score']
stds = grid_result.cv_results_['std_train_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(moy, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyError: ignored

In [61]:
xgb_reg = GradientBoostingRegressor(criterion = 'friedman_mse', learning_rate = 0.01, loss = 'squared_error', max_features = 'sqrt', n_estimators = 800, max_depth = 10) # initialisation du modèle # 'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'squared_error', 'max_features': 'auto', 'n_estimators': 50
xgb_reg.fit(X_train, y_train) # phase d'entraînement 

# Calcul de la performance du Gradient Boosting en cross-validation
cvs = cross_val_score(xgb_reg, X_train, y_train, cv = 5)
print ("Taux moyen de bonne classification (validation croisée) :", cvs.mean())

ValueError: ignored

In [ ]:
y_Pred = xgb_reg.predict(X_test) # phase de prédiction

In [ ]:
y_test = pd.DataFrame(y_test, columns=['price'])
y_Pred = pd.DataFrame(y_Pred, columns=['price'])

print("r2_score :",metrics.r2_score(y_test,y_Pred)) # 
#print("Perte de régression d'erreur logarithmique quadratique moyenne :",metrics.mean_squared_log_error(y_test,y_Pred)) # Perte de régression d'erreur logarithmique quadratique moyenne.
print("rmse :",metrics.mean_squared_error(y_test,y_Pred, squared=False)) # mean scared error

r2_score : 0.9071325177101914
rmse : 26.931412340664508


Sans normalisation : r2 => 0.93 & rmse => 15,895
Avec normalisation : r2 => 0.9069 & rmse => 26,9605

Normalisation des données

In [ ]:
def normalisation(df,liste_var):

  # copy the data
  #new_df = df.copy()
  
  # apply normalization techniques
  for column in liste_var:
      df[column] = (df[column] - df[column].mean()) / df[column].std()    
  
  return df
  # view normalized data   
  #display(df_z_scaled)

In [ ]:
df = normalisation(df,['stock','date'])

# Prédiction sur le jeu de test

In [52]:
df_test = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/df_test.csv')
df_test

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
0,0,1,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,1,1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,2,1,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,3,1,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,4,1,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,6639,843,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,6640,844,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,6641,844,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0
6642,6642,844,vienna,1,irish,1,794,456,0,Yin Yang,Ardisson,1,0,0


In [53]:
# ajout colonne cluster
df_test = df_test.merge(new_df[['hotel_id','cluster']], on='hotel_id', how='left')

# Transformation des variables qualitatives en quantitatives
dummies_language = pd.get_dummies(df_test['language'], prefix= 'Language')
dummies_city = pd.get_dummies(df_test['city'], prefix= 'City')
dummies_group = pd.get_dummies(df_test['group'], prefix= 'Group')
dummies_brand = pd.get_dummies(df_test['brand'], prefix= 'Brand')

df_test = pd.concat([df_test, dummies_language, dummies_city, dummies_group, dummies_brand], axis=1)
df_test.drop(['language', 'city', 'group', 'brand', 'avatar_id'], axis=1, inplace=True)

# normalisation des données
#df_test = normalisation(df_test,['stock','date'])

In [54]:
col = X_train.columns
X_test = df_test[col]

In [ ]:
np.unique(X_train['cluster'])

array([0, 1, 2, 3, 4], dtype=int32)

In [85]:
xgb_reg = GradientBoostingRegressor(n_estimators = 750, criterion = 'friedman_mse', learning_rate = 0.1, loss = 'squared_error', max_features = 'sqrt', max_depth = 4) # initialisation du modèle
xgb_reg.fit(X_train, Y_train) # phase d'entraînement 

# Calcul de la performance du Gradient Boosting en cross-validation
#cvs = cross_val_score(xgb_reg, X_train, Y_train, cv = 5)
#print ("Taux moyen de bonne classification (validation croisée) :", cvs.mean())

GradientBoostingRegressor(max_depth=4, max_features='sqrt', n_estimators=750)

In [86]:
predictions = xgb_reg.predict(X_test) # phase de prédiction

In [87]:
df_test["price"] = predictions

In [88]:
df_test

,index,order_requests,date,mobile,hotel_id,stock,parking,pool,children_policy,cluster,...,Brand_Independant,Brand_J.Halliday Inn,Brand_Marcure,Brand_Morriot,Brand_Navatel,Brand_Quadrupletree,Brand_Royal Lotus,Brand_Safitel,Brand_Tripletree,price
0,0,1,21,0,161,46,1,0,0,0,...,0,1,0,0,0,0,0,0,0,105.078474
1,1,1,21,0,187,32,1,1,0,0,...,0,0,1,0,0,0,0,0,0,135.061267
2,2,1,21,0,279,12,1,0,0,2,...,1,0,0,0,0,0,0,0,0,62.859242
3,3,1,21,0,395,10,0,0,0,2,...,0,0,0,0,0,0,0,0,0,58.590326
4,4,1,21,0,488,42,1,1,1,1,...,0,0,0,0,0,0,0,1,0,208.090113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,6639,843,5,0,987,1,1,0,0,2,...,0,0,0,0,0,0,0,0,0,97.608782
6640,6640,844,1,1,26,1,1,0,0,4,...,0,0,1,0,0,0,0,0,0,247.960125
6641,6641,844,1,1,263,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,147.649099
6642,6642,844,1,1,456,0,1,0,0,4,...,0,0,0,0,0,0,0,0,0,176.179044


In [89]:
# sub30
test_exp = df_test[["index","price"]]
test_exp

,index,price
0,0,105.078474
1,1,135.061267
2,2,62.859242
3,3,58.590326
4,4,208.090113
...,...,...
6639,6639,97.608782
6640,6640,247.960125
6641,6641,147.649099
6642,6642,176.179044


In [66]:
# sub29
test_exp = df_test[["index","price"]]
test_exp

,index,price
0,0,111.572836
1,1,135.904395
2,2,65.105389
3,3,61.514751
4,4,203.287164
...,...,...
6639,6639,89.697527
6640,6640,227.981729
6641,6641,148.241335
6642,6642,182.793107


In [59]:
# sub28
test_exp = df_test[["index","price"]]
test_exp

,index,price
0,0,112.683973
1,1,148.679270
2,2,68.773534
3,3,63.468978
4,4,211.542369
...,...,...
6639,6639,97.396222
6640,6640,223.611479
6641,6641,140.445379
6642,6642,176.535870


In [ ]:
# sub 23
test_exp = df_test[["index","price"]]
test_exp

,index,price
0,0,102.604391
1,1,134.094511
2,2,65.079800
3,3,58.900868
4,4,196.855319
...,...,...
6639,6639,102.298422
6640,6640,242.530165
6641,6641,150.996916
6642,6642,179.007599


In [ ]:
# sub 11
test_exp = df_test[["index","price"]]
test_exp

,index,price
0,0,106.506676
1,1,131.365769
2,2,68.002422
3,3,54.632270
4,4,199.074835
...,...,...
6639,6639,87.066651
6640,6640,257.308022
6641,6641,156.644190
6642,6642,173.582192


In [ ]:
test_exp

,index,price
0,0,102.604391
1,1,134.094511
2,2,65.079800
3,3,58.900868
4,4,196.855319
...,...,...
6639,6639,102.298422
6640,6640,242.530165
6641,6641,150.996916
6642,6642,179.007599


In [99]:
test_exp.to_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/sub30.csv', index=False)

In [100]:
new_df

,index,price
0,0,104.400352
1,1,134.957775
2,2,64.982998
3,3,56.593485
4,4,197.542095
...,...,...
6639,6639,99.728328
6640,6640,243.132230
6641,6641,149.931205
6642,6642,177.857747


INterprétation des prédictions

In [91]:
sub10 = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/SOUMISSIONS/sub10.csv') # 20.19
sub23 = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/SOUMISSIONS/sub23.csv') # 19.78
sub24 = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/SOUMISSIONS/sub24.csv') # 19.56
sub26 = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/SOUMISSIONS/sub26.csv') # 19.16
sub27 = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/SOUMISSIONS/sub27.csv') # 19.26
sub29 = pd.read_csv('/content/drive/MyDrive/M2 SID/DEFI IA 2023/SOUMISSIONS/sub29.csv') # 19.98

In [96]:
df_res = pd.concat([sub23,sub24,sub26,sub27,sub29])

In [95]:
df_res = sub26
df_res = df_res.merge(sub27[['index','price']], on='index', how='left')
df_res = df_res.merge(sub24[['index','price']], on='index', how='left')
df_res = df_res.merge(sub23[['index','price']], on='index', how='left')
df_res = df_res.merge(sub29[['index','price']], on='index', how='left')
df_res

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'price_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,index,price_x,price_y,price_x,price_y,price
0,0,105.336969,104.230822,104.751105,102.604391,105.078474
1,1,134.107024,134.931902,136.594171,134.094511,135.061267
2,2,64.821853,65.513937,66.640159,65.079800,62.859242
3,3,55.198019,56.094275,54.183937,58.900868,58.590326
4,4,198.127537,194.905091,189.732416,196.855319,208.090113
...,...,...,...,...,...,...
6639,6639,96.092351,100.258215,102.383872,102.298422,97.608782
6640,6640,245.167369,241.925256,238.078235,242.530165,247.960125
6641,6641,151.025800,150.501732,149.482479,150.996916,147.649099
6642,6642,177.247481,178.277423,178.577188,179.007599,176.179044


In [97]:
dic = {'index':[],
       'price':[]} # prix moyen

for i in np.unique(df_res['index']):
  dic['index'].append(i)
  dic['price'].append(df_res[df_res.index==i]['price'].mean())

In [98]:
new_df = pd.DataFrame(dic)
new_df

,index,price
0,0,104.400352
1,1,134.957775
2,2,64.982998
3,3,56.593485
4,4,197.542095
...,...,...
6639,6639,99.728328
6640,6640,243.132230
6641,6641,149.931205
6642,6642,177.857747
